In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display_html
import numpy as np

# Source of code: https://stackoverflow.com/questions/28763891/what-should-i-do-when-tr-has-rowspan
import html_table_to_pandas as html_pd

# Football data from wikipedia

The objective of this notebook is to extract information on British football teams from Wikipedia. This primarily utilises the wikipedia API to extract the data, then Beautiful soup is used to manipulate the data.

## Wikipedia API

The documentation for the wikipedia API is here:
https://www.mediawiki.org/wiki/API:Main_page

The inputs to the get request are:
- URL: the wikipedia API endpoint used throughout.
- TITLE: the title of the page to be scraped
- PARAMS: the API takes its input through parameters provided by the query string

In [ ]:
S = requests.Session() # This object allows you to persist certain parameters across requests. Results in better performance.

URL = "https://en.wikipedia.org/w/api.php" # English wikipedia API endpoint

TITLE = "List of football clubs in England"

PARAMS = {
    'action': 'parse', # the action is to parse the content of the page
    'page': TITLE,
    'format': 'json'   # the API can return other formats but they are standardising to JSON
    #'UTF8':            # convert to UTF-8 encoding - does this work? Is it already UTF-8?
}

### Requesting data from a page
Next, we must request the data from the wiki page and turn it into a python object that we can easily manipulate. We must interpret the request data as JSON in order to convert it into __class 'dict'__.

The scraped data are also stored in a JSON file for future use so that we do not need to make the same request multiple times.

In [ ]:
'''
# Get the response data as a python object. 
R = S.get(url=URL, params=PARAMS)
DATA = R.json() # Interpret the request data as json: print(type(DATA)) --> <class 'dict'>

# Write to JSON file
with open('list_clubs_in_england.json', 'w') as f:
    json.dump(DATA, f)
'''

In [2]:
# Read JSON file
with open('list_clubs_in_england.json') as data_file:
    DATA = json.load(data_file)

### The data structure within the dictionary object:

In [ ]:
DATA.keys()

In [ ]:
parse=DATA['parse']
parse.keys()

In [ ]:
text = parse['text']
type(text['*'])

### Manipulating the html

#### Defining the function(s)
Here is the original code where the following functions are defined: https://stackoverflow.com/questions/28763891/what-should-i-do-when-tr-has-rowspan

Very useful when a table has rowspan and colspan.

#### Producing the pandas dataframe

In [3]:
# Parsing the HTML using Beautiful Soup
soup = BeautifulSoup(DATA['parse']['text']['*'], "html.parser")
# print(soup.prettify())

# Extract all of the tables from the soup and put them into separate elements of a list
alltables = soup.findAll('table')
print("Number of tables found : " , len(alltables))

Number of tables found :  31


In [41]:
alltab_parse2 = [html_pd.main(alltables[tab]) for tab in range(1, 25)]
result = pd.concat(alltab_parse2) #.reset_index(drop=True)
result.drop(result[result.index == 0].index)


,0,1,2,3,4
1,A.F.C. Aldermaston,Hellenic League Division One East,10,Atom Men,NaN
2,A.F.C. Blackpool,North West Counties League Division One North,10,Mechanics,NaN
3,A.F.C. Bournemouth,Premier League,1,Cherries,NaN
4,A.F.C. Bridgnorth,West Midlands (Regional) League Premier Division,10,Meadow Men,NaN
5,A.F.C. Croydon Athletic,Southern Counties East League Premier Division,9,Rams,NaN
6,A.F.C. Darwen,North West Counties League Division One North,10,Salmoners,Relegated from North West Counties League Premier
7,A.F.C. Dunstable,Southern League Division One Central,8,Ods,NaN
8,A.F.C. Emley,Northern Counties East League Division One,10,Pewits,NaN
9,A.F.C. Fylde,National League,5,Coasters,NaN
10,A.F.C. Hayes,Combined Counties League Premier Division,9,Brook,NaN


In [42]:
#result = pd.concat(alltab_parse2) #.reset_index(drop=True)
len(result.loc[result.index == 0])

24

In [31]:
#result.keys()
#alltab_parse[1]
result.drop(result.index[0])
result.head()

,0,1,2,3,4
0,Club,League/Division,Lvl,Nickname,Change from 2017–18
1,A.F.C. Aldermaston,Hellenic League Division One East,10,Atom Men,NaN
2,A.F.C. Blackpool,North West Counties League Division One North,10,Mechanics,NaN
3,A.F.C. Bournemouth,Premier League,1,Cherries,NaN
4,A.F.C. Bridgnorth,West Midlands (Regional) League Premier Division,10,Meadow Men,NaN


In [ ]:
# There are 20 teams in the Premier League
prem_teams = result.loc[result['League/Division'] == 'Premier League']
len(prem_teams.index)

## Extract team information on Premier League

In [ ]:
'''
# Extract data on all Premier League teams, convert from JSON to dictionary structure. Store each dictionary structure in a list
club_data_json = [(S.get(url=URL, params={'action': "parse",'page': row.Club,'format': "json"})).json() for row in prem_teams.itertuples()]

# Write to a file. Each teams data is written on a single line
with open('premier_league_club_data.json', 'w') as f:
    json.dump(club_data_json, f)
'''

In [ ]:
with open('premier_league_club_data.json') as f:
    club_data_json = json.load(f)

In [ ]:
# Place the first team page data into bs4
soup_club1 = BeautifulSoup(club_data_json[0]['parse']['text']['*'], "html.parser")

alltables_club1 = soup_club1.findAll("table")

html_content = str(alltables_club1[2])
display_html(html_content, raw=True)

In [ ]:
dfs = pd.read_html(html_content)
dfs[1]

In [ ]:
dfs[2]

#### Club official info

In [ ]:
# Must check the number of columns
club1_officials=parse_html_table(alltables_club1[7], alltables_club1[7])
club1_officials

## Player data

Retrieve data on each individual player.

What kind of data and what format?

### Check for disambiguation page

In [ ]:
TITLE_PLAYER = 'Simon Francis'

PARAMS_DISAMB = {
    'action': 'query', # the action is to parse the content of the page
    'titles': TITLE_PLAYER,
    'format': 'json', # the API can return other formats but they are standardising to JSON
    'prop': 'categories'
}

# Get the response data as a python object. 
R = S.get(url=URL, params=PARAMS_DISAMB)
player_1_query = R.json() # Interpret the request data as json: print(type(DATA)) --> <class 'dict'>

In [ ]:
#  Convert to string in order to easily test if contains specific string
player_1_query = json.dumps(player_1_query)

if 'Category:All disambiguation pages' in player_1_query:
    TITLE_PLAYER = TITLE_PLAYER + ' (footballer)'

In [ ]:
TITLE_PLAYER

### Retrieve player info


In [ ]:
'''
PARAMS_PLAYER = {
    'action': 'parse', # the action is to parse the content of the page
    'page': TITLE_PLAYER,
    'format': 'json' # the API can return other formats but they are standardising to JSON
}


# Get the response data as a python object. 
R = S.get(url=URL, params=PARAMS_PLAYER)
DATA_PLAYER = R.json() # Interpret the request data as json: print(type(DATA)) --> <class 'dict'>

# Write to JSON file
with open('player_data.json', 'w') as f:
    json.dump(DATA_PLAYER, f)
'''

In [ ]:
# Read JSON file
with open('player_data.json') as data_file:
    DATA_PLAYER = json.load(data_file)

In [ ]:
type(DATA_PLAYER)

In [ ]:
# Place the first team page data into bs4
soup_player1 = BeautifulSoup(DATA_PLAYER['parse']['text']['*'], "html.parser")

alltables_player1 = soup_player1.findAll("table")

# Redundent code
#html_content = str(alltables_player1[1])
#fs_player = pd.read_html(html_content, header = 0, tupleize_cols=True, flavor = 'bs4')

In [ ]:
table = alltables_player1[1]
## run the above functions to extract the data
rows, num_rows, num_cols = pre_process_table(table)
df = process_rows(rows, num_rows, num_cols)

# Re-do the formatting
df = df[~df[1].isin(['Total', 'Career total', 'Season'])] # Remove rows containing totals and headers
df = df.drop([11, 12], axis = 1) # Remove columns containing totals

# Re-do the headers
player_cols = ['Club', 'Season','Division', 'League - Apps', 'League - Goals', 'FA Cup - Apps', 'FA Cup - Goals', 'League Cup - Apps', 'League Cup - Goals', 'Other - Apps', 'Other - Goals']
df.columns = player_cols # Apply the correct headers

# Remove link references from 'Season' column
df['Season'] = df['Season'].str.extract(r"^(\d{4}–\d{2})", expand = False)

# TO DO: convert data type of Season column to date/year

df = df.reset_index(drop=True)
df.head()